In [ ]:
from peft import PromptTuningConfig, PromptTuningInit, TaskType, get_peft_model
from transformers import (
    AutoTokenizer,
    DataCollatorForLanguageModeling,
    GPTNeoXForCausalLM
)
from torch.utils.data import DataLoader
import datasets

In [ ]:
dataset = datasets.load_from_disk("/fsx/proj-chemnlp/data/EleutherAI/pythia-160m/marianna13/chemrxiv")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    pretrained_model_name_or_path="EleutherAI/pythia-160m",
    revision="main",
)
tokenizer.add_special_tokens({"pad_token": "<|padding|>"})

In [ ]:
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)
dl = DataLoader(dataset, batch_size=2, collate_fn=data_collator)

In [ ]:
model = GPTNeoXForCausalLM.from_pretrained(
    pretrained_model_name_or_path="EleutherAI/pythia-160m",
    revision="main",
)

In [ ]:
for b in dl:
    token_types = b.pop('token_type_ids')
    break

In [ ]:
out = model(**b)
out['loss']

In [ ]:
peft_config = PromptTuningConfig(
    task_type=TaskType.CAUSAL_LM,
    prompt_tuning_init=PromptTuningInit.TEXT,
    num_virtual_tokens=10,
    prompt_tuning_init_text=" ",
    tokenizer_name_or_path="EleutherAI/pythia-160m",
)
model = get_peft_model(model, peft_config)

In [ ]:
out = model(**b)
out['loss']